# Importing Libraries

In [1]:
import numpy as np
import pandas as pd
import cv2
import os
from skimage.feature import graycomatrix, graycoprops
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from hmmlearn import hmm
from hmmlearn.hmm import GaussianHMM
import joblib
import re
from scipy.fftpack import dct

# Extracting GLCM features

In [2]:
# Function to compute GLCM features for six properties across four directions
def compute_glcm_features_4_directions(window, distances=[1], levels=256):
    # Convert to grayscale if the window is not already
    if len(window.shape) == 3:
        window = cv2.cvtColor(window, cv2.COLOR_BGR2GRAY)
    
    # Ensure the window pixel intensity is in the range [0, levels-1]
    window = np.clip(window, 0, levels - 1).astype(np.uint8)
    
    # Define angles for the four directions: 0°, 45°, 90°, 135°
    angles = [0, np.pi/2]
    
    # Compute GLCM
    glcm = graycomatrix(window, distances=distances, angles=angles, levels=levels, symmetric=True, normed=True)
    
    # Extract GLCM properties for six features
    features = {}
    features['contrast'] = np.mean(graycoprops(glcm, 'contrast'))
    features['dissimilarity'] = np.mean(graycoprops(glcm, 'dissimilarity'))
    features['homogeneity'] = np.mean(graycoprops(glcm, 'homogeneity'))
    features['ASM'] = np.mean(graycoprops(glcm, 'ASM'))  # Angular Second Moment (Energy)
    features['energy'] = np.mean(np.sqrt(graycoprops(glcm, 'ASM')))  # Energy is sqrt(ASM)
    features['correlation'] = np.mean(graycoprops(glcm, 'correlation'))
    
    return [features['contrast'], features['dissimilarity'], features['homogeneity'], 
            features['ASM'], features['energy'], features['correlation']]


# Extracting DCT features

In [3]:
def compute_dct_features(window):
    # Convert to grayscale if the window is not already
    if len(window.shape) == 3:
        window = cv2.cvtColor(window, cv2.COLOR_BGR2GRAY)
    
    # Flatten the window to 1D array for DCT computation
    window_flat = window.flatten()
    
    # Compute DCT and take the first N coefficients (e.g., 198)
    dct_features = dct(window_flat, norm='ortho')[:50]
    
    return dct_features

# Combining both GLCM and DCT features

# Load Data

In [5]:
labels_df = pd.read_excel(r'D:\phd\OneDrive\AMRITA\gt_Window (2).xlsx')  # Update path as needed
image_folder = r'D:\phd\OneDrive\AMRITA\color_window_double1 (1)\color_window_double1'  # Update path as needed


# GLCM feature sequences for each character across all images

In [6]:
def compute_combined_features(window, dct_features=50):
    # Convert to grayscale if needed
    #if len(window.shape) == 3:
       # window = cv2.cvtColor(window, cv2.COLOR_BGR2GRAY)
 
    # Normalize the window
    #window = cv2.normalize(window, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
    # Compute DCT features
    dct_feats = compute_dct_features(window)
    # Compute Wavelet features
    glcm_feature = compute_glcm_features_4_directions(window)
    # Combine DCT and Wavelet features
    combined_features = np.hstack([dct_feats, glcm_feature])
    return combined_features

In [23]:
 len(combined_features)

56

In [8]:
combined_features_glcm_and_dct = {}

window_width = 30   # Width of each sliding window in pixels
step_size = 10     # Step size of the sliding window in pixels

# Process each image (word) in the dataset
for index, row in labels_df.iterrows():
    image_name = row['image name']          # image name of respective gt text
    character_sequence = row['Label']          # gt text
    
    # Load the corresponding image
    image_path = os.path.join(image_folder, image_name)
    image = cv2.imread(image_path)
    if image is None:
        print(f"Image {image_name} could not be loaded.")
        continue

    image_width = image.shape[1]  # Get image width
    
    # Calculate the width of each character region based on the sequence length
    num_characters = len(character_sequence)
    character_width = image_width // num_characters
    
    # Loop through each character in the sequence and collect its GLCM features
    for i, char in enumerate(character_sequence):
        # Define the region corresponding to the current character
        region_start = i * character_width
        region_end = region_start + character_width
        character_region = image[:, region_start:region_end]  # Assume height is all rows
        
        # Split the character region into windows to capture GLCM features
        num_windows = (character_width - window_width) // step_size + 1
        char_combined_sequence = []
        
        for j in range(num_windows):
            # Calculate the start and end of the window within the character region
            window_start = region_start + j * step_size
            window_end = window_start + window_width
            
            # Extract the window
            window = image[:, window_start:window_end]
            
            # Compute GLCM features for this window across four directions
            # glcm_features = compute_glcm_features_4_directions(window)
            # dct_features = compute_dct_features(window)
            # combined_features = np.hstack([glcm_features, dct_features])
            combined_features = compute_combined_features(window)
            # combine_features = combine_glcm_dct_features(dct_features, glcm_features)
            char_combined_sequence.append(combined_features)

        
        # Append this character's GLCM features to the global dictionary
        if char not in combined_features_glcm_and_dct:
            combined_features_glcm_and_dct[char] = []
        combined_features_glcm_and_dct[char].append(char_combined_sequence)

In [24]:
len(char_combined_sequence)

14

# Printing GLCM and DCT features sequences for each character

In [9]:
# Print the number of GLCM feature sequences for each character
for char, sequences in combined_features_glcm_and_dct.items():
    print(f"Character '{char}'  has {len(sequences)}    sequences of GLCM features.")


Character 'െ'  has 618    sequences of GLCM features.
Character 'ത'  has 535    sequences of GLCM features.
Character 'അ'  has 103    sequences of GLCM features.
Character '്'  has 1202    sequences of GLCM features.
Character 'ാ'  has 993    sequences of GLCM features.
Character 'ി'  has 729    sequences of GLCM features.
Character 'പ'  has 339    sequences of GLCM features.
Character 'റ'  has 432    sequences of GLCM features.
Character 'ര'  has 681    sequences of GLCM features.
Character 'ു'  has 455    sequences of GLCM features.
Character 'ള'  has 78    sequences of GLCM features.
Character 'ക'  has 473    sequences of GLCM features.
Character 'ഷ'  has 105    sequences of GLCM features.
Character 'മ'  has 361    sequences of GLCM features.
Character 'ആ'  has 28    sequences of GLCM features.
Character 'ഠ'  has 61    sequences of GLCM features.
Character 'ൽ'  has 40    sequences of GLCM features.
Character 'ർ'  has 62    sequences of GLCM features.
Character 'ഇ'  has 30    sequenc

# Save the GLCM features to a file for analysis


In [11]:
output_path =r'D:\phd\OneDrive\AMRITA\character_glcm_and_dct_features_final.xlsx'

features_df = pd.DataFrame({
    'Character': [char for char, sequences in combined_features_glcm_and_dct.items() for _ in sequences],
    'GLCM Features': [seq for sequences in combined_features_glcm_and_dct.values() for seq in sequences]
})

features_df.to_excel(output_path, index=False)
print(f"GLCM features saved to {output_path}")

GLCM features saved to D:\phd\OneDrive\AMRITA\character_glcm_and_dct_features_final.xlsx


In [12]:
len(char_combined_sequence)

14

# Storing Character HMMs

In [25]:
character_hmms = {}
num_states = 4

# Example: 'character_glcm_sequences' contains the GLCM feature sequences for each character
for char, sequences in combined_features_glcm_and_dct.items():
    # Remove empty sequences
    sequences = [seq for seq in sequences if len(seq) > 0]
    print(len(sequences))
    if len(sequences) == 0:
        print(f"Warning: No valid sequences for character {char}. Skipping this character.")
        continue  # Skip to the next character if no valid sequences
    
    if len(sequences) >= 15:
        # Prepare training data for the HMM
        X = np.vstack(sequences)  # Stack the sequences into a single array
        lengths = [len(seq) for seq in sequences]  # Length of each sequence
        print(lengths)
        # Initialize HMM for this character
        model = hmm.GaussianHMM(n_components=num_states, covariance_type="diag", n_iter=1000, init_params='', verbose=True)


        # Use the custom model
        model.fit(X, lengths)

        # Store the trained model
        character_hmms[char] = model
        print(f"Model trained for character: {char}")

618
[30, 14, 30, 30, 30, 14, 14, 14, 14, 14, 30, 14, 14, 30, 14, 30, 14, 30, 14, 14, 14, 14, 14, 30, 14, 14, 14, 30, 14, 14, 30, 30, 30, 14, 6, 8, 8, 14, 14, 14, 30, 6, 14, 30, 14, 30, 14, 14, 30, 14, 14, 30, 30, 14, 14, 6, 14, 14, 30, 14, 14, 14, 14, 14, 14, 14, 30, 14, 8, 8, 8, 8, 3, 4, 8, 8, 8, 8, 6, 14, 4, 8, 8, 8, 8, 8, 8, 8, 14, 6, 14, 6, 8, 8, 4, 2, 4, 4, 3, 6, 14, 14, 4, 4, 14, 14, 6, 14, 14, 14, 2, 2, 8, 8, 14, 14, 8, 8, 8, 6, 6, 4, 4, 8, 8, 14, 14, 8, 8, 14, 8, 6, 8, 14, 4, 14, 8, 8, 14, 8, 8, 14, 30, 8, 4, 8, 8, 14, 14, 4, 4, 14, 6, 14, 14, 8, 8, 14, 14, 8, 14, 14, 8, 8, 14, 14, 8, 14, 4, 8, 8, 6, 6, 14, 14, 8, 8, 14, 4, 4, 2, 2, 3, 2, 2, 2, 4, 2, 3, 4, 4, 6, 4, 3, 1, 2, 2, 4, 2, 4, 4, 2, 1, 3, 3, 4, 3, 1, 2, 1, 4, 3, 4, 2, 4, 4, 4, 3, 1, 2, 2, 2, 2, 2, 2, 3, 3, 2, 30, 1, 2, 2, 1, 2, 2, 4, 2, 2, 3, 3, 3, 4, 3, 2, 2, 2, 3, 3, 3, 2, 3, 1, 2, 4, 2, 3, 2, 4, 4, 4, 4, 3, 1, 6, 4, 4, 3, 2, 14, 30, 14, 8, 14, 14, 30, 14, 30, 14, 14, 30, 14, 14, 14, 14, 30, 14, 14, 30, 14, 14, 14, 6

         1 -2103689.95085179             +nan
         2 -2059314.14835265  +44375.80249914
         3 -2028554.76083613  +30759.38751653
         4 -2008462.07532153  +20092.68551460
         5 -2001865.34244261   +6596.73287892
         6 -1999359.19343770   +2506.14900491
         7 -1998153.36578282   +1205.82765488
         8 -1997477.60489922    +675.76088361
         9 -1996930.47071562    +547.13418360
        10 -1996645.21937211    +285.25134350
        11 -1996463.87139847    +181.34797364
        12 -1996298.56006403    +165.31133444
        13 -1996195.91749981    +102.64256422
        14 -1996128.33041599     +67.58708382
        15 -1996077.20703522     +51.12338076
        16 -1996045.62801078     +31.57902444
        17 -1996028.42038911     +17.20762167
        18 -1996018.49457578      +9.92581333
        19 -1996012.40192692      +6.09264886
        20 -1996008.33246130      +4.06946562
        21 -1996005.60045280      +2.73200850
        22 -1996003.89604765      

Model trained for character: െ
525
[14, 8, 8, 14, 6, 6, 6, 6, 8, 14, 30, 14, 30, 8, 14, 30, 30, 14, 8, 8, 14, 6, 8, 14, 8, 8, 14, 14, 30, 8, 8, 8, 8, 14, 8, 14, 8, 8, 8, 6, 4, 6, 6, 14, 8, 8, 6, 8, 4, 4, 3, 3, 6, 8, 8, 8, 8, 14, 14, 6, 8, 8, 14, 14, 14, 4, 8, 6, 6, 8, 6, 14, 8, 4, 4, 8, 8, 8, 6, 4, 4, 4, 6, 6, 6, 6, 8, 8, 8, 14, 6, 8, 14, 14, 4, 6, 14, 14, 6, 8, 14, 6, 6, 8, 6, 8, 2, 4, 14, 30, 14, 8, 6, 3, 3, 4, 8, 6, 6, 4, 14, 14, 8, 6, 6, 8, 14, 8, 14, 6, 2, 4, 6, 6, 8, 14, 14, 6, 8, 6, 8, 4, 4, 4, 2, 2, 1, 1, 2, 3, 2, 1, 3, 3, 2, 4, 2, 3, 1, 1, 3, 2, 2, 3, 2, 2, 4, 3, 2, 2, 2, 4, 1, 2, 2, 1, 3, 2, 4, 3, 3, 2, 2, 4, 3, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 1, 1, 2, 2, 2, 2, 2, 3, 3, 14, 1, 2, 2, 2, 4, 2, 2, 2, 4, 4, 4, 1, 3, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 3, 2, 2, 2, 1, 1, 2, 2, 3, 2, 2, 3, 2, 3, 2, 3, 2, 6, 3, 3, 2, 2, 2, 4, 8, 4, 3, 2, 3, 3, 3, 2, 2, 1, 1, 2, 2, 2, 2, 2, 1, 2, 1, 2, 2, 2, 1, 1, 2, 2, 2, 1, 3, 2, 1, 2, 1, 1, 6, 4, 3, 4, 2, 2, 3, 6, 6, 14, 8, 14, 8, 6, 6, 6, 14, 14, 14, 6, 6

         1 -1027759.20600303             +nan
         2 -1004918.50881586  +22840.69718717
         3 -990255.87536713  +14662.63344873
         4 -984608.74121296   +5647.13415417
         5 -983985.20307593    +623.53813704
         6 -983826.28415019    +158.91892574
         7 -983722.35314979    +103.93100041
         8 -983634.90054659     +87.45260319
         9 -983587.29467086     +47.60587574
        10 -983544.39851462     +42.89615623
        11 -983515.24210918     +29.15640544
        12 -983503.51752133     +11.72458784
        13 -983494.28131842      +9.23620291
        14 -983486.59237276      +7.68894567
        15 -983479.46148326      +7.13088950
        16 -983471.90281839      +7.55866486
        17 -983467.23800614      +4.66481225
        18 -983464.44845459      +2.78955156
        19 -983462.82159901      +1.62685557
        20 -983461.91673794      +0.90486107
        21 -983461.42119189      +0.49554606
        22 -983461.14215306      +0.27903882
        

Model trained for character: ത
102
[30, 30, 30, 4, 30, 30, 14, 8, 30, 30, 14, 14, 14, 4, 4, 3, 2, 30, 14, 14, 30, 14, 14, 30, 14, 14, 30, 14, 14, 4, 14, 14, 14, 14, 14, 14, 6, 14, 8, 4, 30, 14, 14, 30, 14, 8, 14, 14, 8, 14, 14, 8, 30, 30, 8, 14, 6, 14, 4, 14, 6, 14, 8, 14, 14, 14, 14, 14, 14, 14, 14, 4, 6, 14, 14, 4, 4, 14, 30, 14, 14, 6, 14, 6, 14, 14, 14, 8, 8, 8, 14, 8, 8, 14, 8, 8, 30, 6, 4, 6, 6, 14]


...

         4  -94482.63947824    +407.17112540
         5  -94390.34572463     +92.29375361
         6  -94377.66759883     +12.67812580
         7  -94364.16198127     +13.50561756
         8  -94355.74592539      +8.41605588
         9  -94353.59655266      +2.14937273
        10  -94352.20417763      +1.39237503
        11  -94346.68722218      +5.51695545
        12  -94328.25515173     +18.43207046
        13  -94313.11031825     +15.14483348
        14  -94287.53600129     +25.57431696
        15  -94287.38561985      +0.15038144
        16  -94287.38646362      -0.00084378
Model is not converging.  Current: -94287.38646362242 is not greater than -94287.38561984604. Delta is -0.0008437763754045591
D:\anaconda\Lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=2.
  warnings.warn(
         1 -10

Model trained for character: ൃ
8
12
3
54
[14, 14, 14, 14, 14, 14, 3, 3, 4, 6, 3, 2, 4, 3, 4, 6, 4, 4, 4, 6, 4, 4, 3, 2, 3, 4, 6, 3, 6, 2, 4, 3, 6, 3, 3, 2, 4, 6, 4, 6, 2, 6, 4, 6, 2, 6, 6, 14, 14, 14, 14, 14, 14, 14]
Model trained for character: ൊ
5
22
[6, 8, 2, 2, 2, 2, 2, 3, 3, 2, 2, 2, 2, 6, 6, 6, 6, 6, 8, 14, 14, 14]


         1  -36011.44031353             +nan
         2  -34934.89063332   +1076.54968021
         3  -34396.24297601    +538.64765732
         4  -34242.10154858    +154.14142743
         5  -34221.22529145     +20.87625713
         6  -34196.17899245     +25.04629900
         7  -34190.71518630      +5.46380615
         8  -34190.69944793      +0.01573837
         9  -34190.69816871      +0.00127922
D:\anaconda\Lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
         1  -45019.06288444             +nan
         2  -44110.32774422    +908.73514022
         3  -43916.24824469    +194.07949953
         4  -43871.66966096     +44.57858373
         5  -43817.78613600     +53.88352495
         6  -43794.29841708     +23.48771893
         7  -43789.71970930      +4.57870778
     

Model trained for character: ഞ
1
26
[8, 14, 8, 4, 4, 3, 1, 3, 4, 6, 4, 6, 6, 6, 6, 8, 6, 6, 6, 6, 6, 8, 6, 6, 4, 6]
Model trained for character: ബ
4
25
[2, 2, 4, 2, 2, 4, 4, 4, 3, 3, 2, 3, 2, 3, 3, 1, 1, 1, 1, 1, 4, 4, 2, 2, 2]
Model trained for character: ഃ
11
3
11
1
2
3
2
1
4


         1  -18016.90391305             +nan
         2  -17168.15230026    +848.75161279
         3  -17152.72000772     +15.43229254
         4  -17152.22862781      +0.49137991
         5  -17152.22862646      +0.00000135


# View the transition matrix after training

In [50]:
print(model.transmat_)  
row_sums = model.transmat_.sum(axis=1)
print(row_sums)

[[1.00000000e+000 1.18417596e-017 2.37499200e-224 8.18469384e-050]
 [7.88823588e-039 1.00000000e+000 6.00118939e-164 9.46507296e-046]
 [8.20262840e-228 2.02135341e-180 1.00000000e+000 1.00679451e-221]
 [1.43830647e-053 5.04952374e-025 6.35480655e-231 1.00000000e+000]]
[1. 1. 1. 1.]


# Save the trained HMM model to a file 

In [51]:
from joblib import dump

# Assuming your models are in a dictionary called `character_hmms`
# e.g., `character_hmms = {'െ': trained_model_1, 'ത': trained_model_2, ... }

def sanitize_filename(char):
    # Replace invalid filename characters with an underscore or remove them
    return re.sub(r'[<>:"/\\|?*]', '_', char)

# Ensure the directory for the models exists
model_directory = '//home//jaykishor_c//ml//Model_combined'
os.makedirs(model_directory, exist_ok=True)

for char, model in character_hmms.items():
    # Sanitize character to create a valid filename
    sanitized_char = sanitize_filename(char)
    print(char, sanitized_char)
    # Save the trained HMM model to a file with the sanitized name in the 'model' directory
    model_path = os.path.join(model_directory, f"{sanitized_char}_hmm.pkl")
    dump(model, model_path)


െ െ
ത ത
അ അ
് ്
ാ ാ
ി ി
പ പ
റ റ
ര ര
ു ു
ള ള
ക ക
ഷ ഷ
മ മ
ആ ആ
ഠ ഠ
ൽ ൽ
ർ ർ
ഇ ഇ
ങ ങ
ന ന
ജ ജ
ട ട
വ വ
ഹ ഹ
ം ം
യ യ
" _
ധ ധ
ച ച
എ എ
ല ല
ഴ ഴ
ദ ദ
ഗ ഗ
ഉ ഉ
ണ ണ
ൂ ൂ
ൻ ൻ
സ സ
ീ ീ
ഭ ഭ
ഖ ഖ
ശ ശ
ൈ ൈ
ഥ ഥ
ൾ ൾ
ഘ ഘ
ൃ ൃ
ൊ ൊ
ഞ ഞ
ബ ബ
ഃ ഃ


In [52]:
print(X.shape)  

(62, 56)


# Testing shell 1

In [32]:
import joblib
from hmmlearn import hmm
import cv2
import numpy as np
import pandas as pd
 
 
# Function to compute the actual sequence based on window mapping
# Function to extract the actual sequence for a specific image from the DataFrame
def get_actual_sequence_from_df(image_name, line_image, label_df, window_width=30, step_size=10):
    # Find the corresponding row in the DataFrame
    row = label_df[label_df['image name'] == image_name]
    if row.empty:
        raise ValueError(f"Image name '{image_name}' not found in the labels DataFrame.")
    # Extract the ground truth character sequence
    character_sequence = row['Label'].values[0]  # Adjust column name if necessary
    # Compute the actual sequence based on ground truth and image dimensions
    actual_sequence = []
    image_width = line_image.shape[1]
    num_characters = len(character_sequence)
    character_width = image_width // num_characters
 
    for i, char in enumerate(character_sequence):
        # Define the region corresponding to this character
        region_start = i * character_width
        region_end = region_start + character_width
        character_region = line_image[:, region_start:region_end]
 
        # Divide the character region into windows
        num_windows = (character_width - window_width) // step_size + 1
        for _ in range(num_windows):
            actual_sequence.append(char)  # Map each window to the current character
    return actual_sequence
# Function to predict the sequence based on HMM models
def predict_line_sequence(line_image, window_width=30, step_size=10):
    predictions = []
    image_width = line_image.shape[1]
    num_windows = (image_width - window_width) // step_size + 1
 
    for i in range(num_windows):
        window_start = i * step_size
        window_end = window_start + window_width
        window = line_image[:, window_start:window_end]
        test_features = compute_combined_features(window)
            # combine_features = combine_glcm_dct_features(dct_features, glcm_features)
          # Convert list to Numpy array if necessary

        #gabor_features = compute_gabor_features(window)
 
            # Combine features (concatenate DCT and Gabor features)
        #combined_features = np.concatenate((dct_features, gabor_features))
 
        test_features = test_features.reshape(1, -1) 

 
        # Calculate likelihoods for each character model
        best_char = None
        best_score = float('-inf')
 
        for char, model in character_hmms.items():
            try:
                score = model.score(test_features )
                if score > best_score:
                    best_score = score
                    best_char = char
            except:
                pass  # Ignore errors for invalid model scoring
 
        if best_char is not None:
            predictions.append(best_char)
 
    return ''.join(predictions)
 
# Test image details
# Load the labels DataFrame
label_file = r'D:\phd\OneDrive\AMRITA\gt_Window (2).xlsx'
label_df = pd.read_excel(label_file)

# Test image details
test_image_path = r"D:\phd\OneDrive\AMRITA\color_window_double1 (1)\color_window_double1//MaI12_Page100_line_2.jpg_window_15.jpg"
test_image_name = test_image_path.split("/")[-1]  # Correctly extracts the image name by splitting on '/'

# print("test image name : ", test_image_name)
# print("Data frame image name : ", label_df['image name'])
 
# Load the test image
test_image = cv2.imread(test_image_path, cv2.IMREAD_GRAYSCALE)
 
# Extract the actual sequence from the DataFrame
actual_sequence = get_actual_sequence_from_df(test_image_name, test_image, label_df)
 
# Predict the sequence using the HMM models
predicted_sequence = predict_line_sequence(test_image)
 
# Print the sequences for comparison
print(f"Actual sequence: {''.join(actual_sequence)}")
print(f"Predicted sequence: {predicted_sequence}")

Actual sequence: ാാാാാാാാാാാാാാാാാാാാാാാാാാാാാാ
Predicted sequence: ിങിിിടാതതതതാാാാടടാാാാാററെിിിെആ


# Extract the character names from the character_hmms dictionary

In [33]:
char_names = list(character_hmms.keys())

# Print the character names
print("Character names:", char_names)

Character names: ['െ', 'ത', 'അ', '്', 'ാ', 'ി', 'പ', 'റ', 'ര', 'ു', 'ള', 'ക', 'ഷ', 'മ', 'ആ', 'ഠ', 'ൽ', 'ർ', 'ഇ', 'ങ', 'ന', 'ജ', 'ട', 'വ', 'ഹ', 'ം', 'യ', '"', 'ധ', 'ച', 'എ', 'ല', 'ഴ', 'ദ', 'ഗ', 'ഉ', 'ണ', 'ൂ', 'ൻ', 'സ', 'ീ', 'ഭ', 'ഖ', 'ശ', 'ൈ', 'ഥ', 'ൾ', 'ഘ', 'ൃ', 'ൊ', 'ഞ', 'ബ', 'ഃ']


In [34]:
len(char_names)

53

In [35]:
ground_truth =char_names

In [36]:
print("Ground truth list:", ground_truth)

Ground truth list: ['െ', 'ത', 'അ', '്', 'ാ', 'ി', 'പ', 'റ', 'ര', 'ു', 'ള', 'ക', 'ഷ', 'മ', 'ആ', 'ഠ', 'ൽ', 'ർ', 'ഇ', 'ങ', 'ന', 'ജ', 'ട', 'വ', 'ഹ', 'ം', 'യ', '"', 'ധ', 'ച', 'എ', 'ല', 'ഴ', 'ദ', 'ഗ', 'ഉ', 'ണ', 'ൂ', 'ൻ', 'സ', 'ീ', 'ഭ', 'ഖ', 'ശ', 'ൈ', 'ഥ', 'ൾ', 'ഘ', 'ൃ', 'ൊ', 'ഞ', 'ബ', 'ഃ']


# Converting 'character to state' and 'state to character'

In [37]:
unique_chars = sorted(set("".join(ground_truth)))
char_to_state = {char: idx for idx, char in enumerate(unique_chars)}
state_to_char = {idx: char for char, idx in char_to_state.items()}

# Testing shel 2

In [38]:
import os
import cv2
import pandas as pd
from nltk.translate.bleu_score import sentence_bleu
 
# Function to calculate BLEU score
def calculate_bleu_score(actual_sequence, predicted_sequence):
    reference = [list(actual_sequence)]  # Wrap in another list for multiple references
    hypothesis = list(predicted_sequence)
    bleu_score = sentence_bleu(reference, hypothesis)
    return bleu_score
 
# Function to calculate accuracy
def calculate_accuracy(actual_sequence, predicted_sequence):
    # Ensure the sequences are the same length for comparison
    min_length = min(len(actual_sequence), len(predicted_sequence))
    if min_length == 0:
        return 0  # Return 0 if either sequence is empty to avoid division by zero
    correct_predictions = sum(1 for i in range(min_length) if actual_sequence[i] == predicted_sequence[i])
    accuracy = (correct_predictions / min_length) * 100  # Calculate accuracy as a percentage
    return accuracy

 
# Folder paths and files
image_folder =r"D:\phd\OneDrive\AMRITA\color_window_double1 (1)\color_window_double1"
label_file = r'D:\phd\OneDrive\AMRITA\gt_Window (2).xlsx'
output_file = r"/\phd\OneDrive\AMRITA\BLEU_Scores_Accuracy_actual_and_prdicated_char_forCombinedfeaturs.xlsx"
 
# Load the labels DataFrame
label_df = pd.read_excel(label_file)
 
# List all image files in the folder
image_files = [f for f in os.listdir(image_folder) if f.endswith(('.jpg', '.png'))]
 
# Initialize a results list and variable to track total accuracy
results = []
total_accuracy = 0
num_images = 0
 
# Process each image
for image_name in image_files:
    image_path = os.path.join(image_folder, image_name)
    # Load the image
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if image is None:
        print(f"Error loading image: {image_name}")
        continue
 
    # Extract the actual sequence from the DataFrame
    actual_sequence = get_actual_sequence_from_df(image_name, image, label_df)
    if actual_sequence is None:
        print(f"No label found for image: {image_name}")
        continue
 
    # Predict the sequence using the HMM models
    predicted_sequence = predict_line_sequence(image)
    # Calculate BLEU score
    bleu_score = calculate_bleu_score(actual_sequence, predicted_sequence)
    # Calculate accuracy
    accuracy = calculate_accuracy(actual_sequence, predicted_sequence)
    print(image_name, accuracy)
    # Add the accuracy to the total
    total_accuracy += accuracy
    num_images += 1
    # Append results
    results.append({
        "Image Name": image_name,
        "Actual Sequence": ''.join(actual_sequence),
        "Predicted Sequence": predicted_sequence,
        #"BLEU Score": bleu_score,
        "Accuracy": accuracy
    })
 
# Calculate the average accuracy
average_accuracy = total_accuracy / num_images if num_images > 0 else 0
 
# Save the results to an Excel sheet
results_df = pd.DataFrame(results)
results_df.to_excel(output_file, index=False)
 
# Print average accuracy
print(f"Average Accuracy: {average_accuracy:.2f}%")
print(f"Results saved to {output_file}")

D:\anaconda\Lib\site-packages\nltk\translate\bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
D:\anaconda\Lib\site-packages\nltk\translate\bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


MaI12_Page100_line_1.jpg_window_0.jpg 6.666666666666667


D:\anaconda\Lib\site-packages\nltk\translate\bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
D:\anaconda\Lib\site-packages\nltk\translate\bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


MaI12_Page100_line_1.jpg_window_1.jpg 3.571428571428571


D:\anaconda\Lib\site-packages\nltk\translate\bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
D:\anaconda\Lib\site-packages\nltk\translate\bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


...

D:\anaconda\Lib\site-packages\nltk\translate\bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
D:\anaconda\Lib\site-packages\nltk\translate\bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


maI16_01_1.jpg_window_7.jpg 6.25


D:\anaconda\Lib\site-packages\nltk\translate\bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
D:\anaconda\Lib\site-packages\nltk\translate\bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
D:\anaconda\Lib\site-packages\nltk\translate\bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


maI16_01_1.jpg_window_8.jpg 0.0
maI16_01_1.jpg_window_9.jpg 0


ValueError: Image name 'maI16_01_10.jpg_window_0.jpg' not found in the labels DataFrame.

# BLUE score for each character

In [ ]:
from nltk.translate.bleu_score import sentence_bleu

characters_to_remove = ['ണ',' ൾ','ു','ഥ','ൃ','ബ','ൂഃ','ഃ','ൻ']
 
# Calculate BLEU score for predictions
for char, model in character_hmms.items():
    try:
        sequences = combined_features_glcm_and_dct[char]
        # Remove empty sequences
        sequences = [seq for seq in sequences if len(seq) > 0]
        # Check if we have any sequences left
        if len(sequences) >= 15:
            X = np.vstack(sequences)
            lengths = [len(seq) for seq in sequences]
            # Check for valid transition matrix
            row_sums = model.transmat_.sum(axis=1)
            if not np.allclose(row_sums, 1):
                print(f"Problem with transition matrix for character '{char}': row sums = {row_sums}")
                characters_to_remove.append(char)
                continue  # Skip this model and move to the next one
            # Get predicted states
            predicted_states = model.predict(X)
            mapped_predictions = [state_to_char[state] for state in predicted_states]
            # Calculate BLEU score
            bleu_score = sentence_bleu([ground_truth], mapped_predictions)
            print(f"BLEU Score for character '{char}': {bleu_score}")
    except ValueError as e:
        print(f"Error with character '{char}': {e}")

BLEU Score for character 'െ': 3.7543342543670055e-232
BLEU Score for character 'ത': 4.5786155309057406e-232
BLEU Score for character 'അ': 5.4269696562331364e-232
BLEU Score for character '്': 5.0806376019353144e-232
BLEU Score for character 'ാ': 3.961047620822523e-232
BLEU Score for character 'ി': 4.614309862807827e-232
BLEU Score for character 'പ': 5.2365941639833255e-232
BLEU Score for character 'റ': 4.879980276268283e-232
BLEU Score for character 'ര': 4.4943550935417704e-232
BLEU Score for character 'ു': 5.569381294996544e-232
BLEU Score for character 'ള': 6.573479617511883e-232
BLEU Score for character 'ക': 4.902213108167789e-232
Problem with transition matrix for character 'ഷ': row sums = [1. 0. 1. 1.]
BLEU Score for character 'മ': 5.165360761805455e-232
BLEU Score for character 'ആ': 6.94876893293765e-232
BLEU Score for character 'ഠ': 6.3757479263969885e-232
BLEU Score for character 'ൽ': 7.50359907734765e-232
BLEU Score for character 'ർ': 7.973824942193111e-232
BLEU Score for char

/home/jaykishor_c/ml/JaykishorEnv/lib/python3.10/site-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/home/jaykishor_c/ml/JaykishorEnv/lib/python3.10/site-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/home/jaykishor_c/ml/JaykishorEnv/lib/python3.10/site-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider 

# Remove characters if they exist in the dictionary

In [ ]:
for char in characters_to_remove:
    print (char)
    if char in character_hmms:
        del character_hmms[char]
        print(f"Deleted HMM model for character: {char}")
    else:
        print(f"Character {char} not found in character_hmms.")

ണ
Deleted HMM model for character: ണ
 ൾ
Character  ൾ not found in character_hmms.
ു
Deleted HMM model for character: ു
ഥ
Character ഥ not found in character_hmms.
ൃ
Deleted HMM model for character: ൃ
ബ
Deleted HMM model for character: ബ
ൂഃ
Character ൂഃ not found in character_hmms.
ഃ
Character ഃ not found in character_hmms.
ൻ
Deleted HMM model for character: ൻ
ഷ
Deleted HMM model for character: ഷ
ട
Deleted HMM model for character: ട
സ
Deleted HMM model for character: സ
ഭ
Deleted HMM model for character: ഭ
ശ
Deleted HMM model for character: ശ
ബ
Character ബ not found in character_hmms.


# Calculate BLEU score for predictions

In [ ]:
# Calculate BLEU score for predictions (this is more useful for sequence generation tasks)
for char, model in character_hmms.items():
    sequences = combined_features_glcm_and_dct[char]
    
    # Remove empty sequences
    sequences = [seq for seq in sequences if len(seq) > 0]
    
    # Check if we have any sequences left
    if len(sequences) >=15:
       
        X = np.vstack(sequences)
    
    lengths = [len(seq) for seq in sequences]
    
    # Get predicted states
    predicted_states = model.predict(X)
    mapped_predictions = [state_to_char[state] for state in predicted_states]

# Calculate BLEU score
bleu_score = sentence_bleu([ground_truth], mapped_predictions)
print(f"BLEU Score: {bleu_score}")

BLEU Score: 8.094051362019983e-232


/home/jaykishor_c/ml/JaykishorEnv/lib/python3.10/site-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/home/jaykishor_c/ml/JaykishorEnv/lib/python3.10/site-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/home/jaykishor_c/ml/JaykishorEnv/lib/python3.10/site-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider 

# Save the BLUR score to an Excel file

In [ ]:
import pandas as pd
from nltk.translate.bleu_score import sentence_bleu
import numpy as np

# List to store results
results = []

# Loop through each character and its model
for char, model in character_hmms.items():
    sequences = combined_features_glcm_and_dct[char]
    
    # Remove empty sequences
    sequences = [seq for seq in sequences if len(seq) > 0]
    
    # Check if we have enough sequences to proceed
    if len(sequences) >= 15:
        X = np.vstack(sequences)

        lengths = [len(seq) for seq in sequences]
        
        # Get predicted states and map to characters
        predicted_states = model.predict(X)
        mapped_predictions = [state_to_char[state] for state in predicted_states]
        
        # Calculate BLEU score
        bleu_score = sentence_bleu([ground_truth], mapped_predictions)
        
        # Store the results
        results.append({
            'Character': char,
            'Mapped Predictions': ''.join(mapped_predictions),
            'BLEU Score': bleu_score
        })

# Create a DataFrame to store results
results_df = pd.DataFrame(results)


# Save the results to an Excel file
output_path = r'//home//jaykishor_c//ml//Output2//mapped_predictions_bleu_scores.xlsx'
results_df.to_excel(output_path, index=False)
print(f"Results saved to {output_path}")

Results saved to mapped_predictions_bleu_scores_hog.xlsx


/home/jaykishor_c/ml/JaykishorEnv/lib/python3.10/site-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/home/jaykishor_c/ml/JaykishorEnv/lib/python3.10/site-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/home/jaykishor_c/ml/JaykishorEnv/lib/python3.10/site-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider 

In [ ]:
results_df

,Character,Mapped Predictions,BLEU Score
0,െ,ആആആആആആആആആആആആആആആആആആആആആആആആആആആആആആആആആആആആആആആആആആആആആആ...,3.754334e-232
1,ത,അഅഅഅഅഅഅഅഅഅഅഅഅഅഅഅഅഅഅഅഅഅഅഅഅഅഅഅഅഅഅഅഅഅഅഅഅഅഅഅഅഅഅഅഅഅ...,4.578616e-232
2,അ,ആആആആആആആആആആആആആആആഅഅഅആആആആആആആആആആആആആആആആആആഅഅഅആആആആആആആ...,5.426970e-232
3,്,"അഅഅഅഅഅഅഅഅഅഅഅഅഅഅഅഅഅഅഅഅഅഅഅഅഅഅഅഅഅഅഅഅഅ""അഅഅഅഅഅഅഅഅഅഅ...",5.080638e-232
4,ാ,ംംംംംംംംംംംംംംംംംംംംംംംംംംംംംംംംംംംംംംംംംംംംംം...,3.961048e-232
5,ി,"ആആആ""""""""""""""""""""ആആആആആആആആആആആആആആ""""""""""""ആആആആആആആആആആആആആ...",4.614310e-232
6,പ,അഅഅഅഅഅഅഅഅഅഅഅഅഅഅഅഅഅഅഅഅഅഅഅഅഅഅഅഅഅഅഅഅഅഅഅഅഅഅഅഅഅഅഅഅഅ...,5.236594e-232
7,റ,"""ആ""ആ""ആ""ആ""ആ""ആ""ആ""ആ""ആ""ആ""ആ""ആ""ആ""ആ""ആ""ആ""ആ""ആ""ആ""ആ""ആ""ആംഅ...",4.879980e-232
8,ര,"ംംംംംംംംംംംം""""""""""""""""""ംംംംം""ംംംംംം""""""""""""""""""""""""""...",4.494355e-232
9,ള,ംംംംംംംംംംംംംംംംംംംംംംംംംംംഅഅഅഅഅഅആആആആആആആആആആആആആ...,6.573480e-232


# Calculate Probability

**1. Log Probability**

In [ ]:
'''for char, model in character_hmms.items():
    sequences = combined_features_glcm_and_dct[char]
    sequences = [seq for seq in sequences if len(seq) > 0]

    if len(sequences) >= 15:
        X = np.vstack(sequences)
        lengths = [len(seq) for seq in sequences]
        log_probability = model.score(X, lengths)
        print(f"Log Probability for character '{char}': {log_probability}")
'''

'for char, model in character_hmms.items():\n    sequences = character_glcm_sequences[char]\n    sequences = [seq for seq in sequences if len(seq) > 0]\n\n    if len(sequences) >= 15:\n        X = np.vstack(sequences)\n        lengths = [len(seq) for seq in sequences]\n        log_probability = model.score(X, lengths)\n        print(f"Log Probability for character \'{char}\': {log_probability}")\n'

**2. Model Probability**

In [ ]:
'''for char, model in character_hmms.items():
    sequences = combined_features_glcm_and_dct[char]
    sequences = [seq for seq in sequences if len(seq) > 0]

    if len(sequences) >= 15:
        X = np.vstack(sequences)
        lengths = [len(seq) for seq in sequences]
        log_probability = model.score(X, lengths)
        model_probability = np.exp(log_probability)  # Convert to regular probability
        print(f"Model Probability for character '{char}': {model_probability}")
'''

'for char, model in character_hmms.items():\n    sequences = character_glcm_sequences[char]\n    sequences = [seq for seq in sequences if len(seq) > 0]\n\n    if len(sequences) >= 15:\n        X = np.vstack(sequences)\n        lengths = [len(seq) for seq in sequences]\n        log_probability = model.score(X, lengths)\n        model_probability = np.exp(log_probability)  # Convert to regular probability\n        print(f"Model Probability for character \'{char}\': {model_probability}")\n'

**3. Transition Matrix Validation**

In [ ]:
'''for char, model in character_hmms.items():
    row_sums = model.transmat_.sum(axis=1)
    if not np.allclose(row_sums, 1):
        print(f"Transition matrix issue for '{char}': {row_sums}")
    else:
        print(f"Transition matrix for '{char}' is valid.")
'''

'for char, model in character_hmms.items():\n    row_sums = model.transmat_.sum(axis=1)\n    if not np.allclose(row_sums, 1):\n        print(f"Transition matrix issue for \'{char}\': {row_sums}")\n    else:\n        print(f"Transition matrix for \'{char}\' is valid.")\n'

**4. Emission Probabilities**

In [ ]:
'''for char, model in character_hmms.items():
    print(f"Emission means for character '{char}': {model.means_}")
    print(f"Emission covariances for character '{char}': {model.covars_}")
'''

'for char, model in character_hmms.items():\n    print(f"Emission means for character \'{char}\': {model.means_}")\n    print(f"Emission covariances for character \'{char}\': {model.covars_}")\n'

**5. Per-State Log Probability**

In [ ]:
'''for char, model in character_hmms.items():
    sequences = combined_features_glcm_and_dct[char]
    sequences = [seq for seq in sequences if len(seq) > 0]

    if len(sequences) >= 15:
        X = np.vstack(sequences)  # Combine all sequences into a single array

        # Use score_samples to compute posterior probabilities for states
        log_prob, posteriors = model.score_samples(X)
        print(f"Log probabilities for character '{char}': {log_prob}")
        print(f"Posterior probabilities for states:\n{posteriors}")
'''

'for char, model in character_hmms.items():\n    sequences = character_glcm_sequences[char]\n    sequences = [seq for seq in sequences if len(seq) > 0]\n\n    if len(sequences) >= 15:\n        X = np.vstack(sequences)  # Combine all sequences into a single array\n\n        # Use score_samples to compute posterior probabilities for states\n        log_prob, posteriors = model.score_samples(X)\n        print(f"Log probabilities for character \'{char}\': {log_prob}")\n        print(f"Posterior probabilities for states:\n{posteriors}")\n'

# Actual and Prediction of character

In [ ]:
# Dictionary to store actual vs predicted characters
results = {'Actual': [], 'Predicted': []}
 
# Process each test character and its sequences
for actual_char, test_sequences in combined_features_glcm_and_dct.items():
    for test_sequence in test_sequences:
        test_sequence = np.array(test_sequence)
        # Dictionary to store scores for each character model
        scores = {}
        # Score the test sequence against each character's HMM model
        for char, model in character_hmms.items():
            try:
                scores[char] = model.score(test_sequence)
            except:
                scores[char] = -np.inf  # If model can't score, assign a low score
        # Predict the character with the highest score
        predicted_char = max(scores, key=scores.get)
        # Store the result
        results['Actual'].append(actual_char)
        results['Predicted'].append(predicted_char)
 
# Save the results to an Excel file
output_path = r'/home/jaykishor_c/ml/Output2//actual_vs_predicted.xlsx'
results_df = pd.DataFrame(results)
results_df.to_excel(output_path, index=False)


print(f"Results saved to {output_path}")

Results saved to actual_vs_predicted.xlsx


# Calculate Accuracy

In [ ]:

correct_predictions = sum([1 for actual, predicted in zip(results['Actual'], results['Predicted']) if actual == predicted])
total_predictions = len(results['Actual'])
accuracy = (correct_predictions / total_predictions) * 100

# Print accuracy
print(f"Accuracy: {accuracy:.2f}%")


Accuracy: 6.58%
